In [2]:
!pip list | grep larch

larch                         5.5.5


In [64]:
import os
import numpy as np
import pandas as pd
import larch
import larch.exampville
from larch import P, X

# Exampville Mode Choice

Discrete choice modeling is at the heart of many transportion planning models.
In this example, we will examine the development of a mode choice model for 
Exampville, an entirely fictional town built for the express purpose of 
demostrating the use of discrete choice modeling tools for transportation 
planning.

In [65]:
skims = larch.OMX(larch.exampville.files.skims, mode='r')

In [66]:
skims

<larch.OMX> ⋯/exampville_skims.omx
 |  shape:(40, 40)
 |  data:
 |    AUTO_COST    (float64)
 |    AUTO_DIST    (float64)
 |    AUTO_TIME    (float64)
 |    BIKE_TIME    (float64)
 |    TRANSIT_FARE (float64)
 |    TRANSIT_IVTT (float64)
 |    TRANSIT_OVTT (float64)
 |    WALK_DIST    (float64)
 |    WALK_TIME    (float64)
 |  lookup:
 |    TAZ_AREA_TYPE (40 |S3)
 |    TAZ_ID        (40 int64)

In [67]:
hh = pd.read_csv(larch.exampville.files.hh)
pp = pd.read_csv(larch.exampville.files.person)
tour = pd.read_csv(larch.exampville.files.tour)

In [68]:
hh.shape, pp.shape, tour.shape

((5000, 13), (12349, 12), (20739, 11))

In [69]:
hh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   X            5000 non-null   float64
 1   Y            5000 non-null   float64
 2   INCOME       5000 non-null   float64
 3   N_VEHICLES   5000 non-null   int64  
 4   HHSIZE       5000 non-null   int64  
 5   geometry     5000 non-null   object 
 6   HOMETAZ      5000 non-null   int64  
 7   HHID         5000 non-null   int64  
 8   N_TRIPS      5000 non-null   int64  
 9   N_TRIPS_HBW  5000 non-null   int64  
 10  N_TRIPS_HBO  5000 non-null   int64  
 11  N_TRIPS_NHB  5000 non-null   int64  
 12  N_WORKERS    5000 non-null   int64  
dtypes: float64(3), int64(9), object(1)
memory usage: 507.9+ KB


In [70]:
pp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12349 entries, 0 to 12348
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   PERSONID       12349 non-null  int64
 1   HHID           12349 non-null  int64
 2   HHIDX          12349 non-null  int64
 3   AGE            12349 non-null  int64
 4   WORKS          12349 non-null  int64
 5   N_WORK_TOURS   12349 non-null  int64
 6   N_OTHER_TOURS  12349 non-null  int64
 7   N_TOURS        12349 non-null  int64
 8   N_TRIPS        12349 non-null  int64
 9   N_TRIPS_HBW    12349 non-null  int64
 10  N_TRIPS_HBO    12349 non-null  int64
 11  N_TRIPS_NHB    12349 non-null  int64
dtypes: int64(12)
memory usage: 1.1 MB


In [71]:
tour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20739 entries, 0 to 20738
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   TOURID       20739 non-null  int64
 1   HHID         20739 non-null  int64
 2   PERSONID     20739 non-null  int64
 3   DTAZ         20739 non-null  int64
 4   TOURMODE     20739 non-null  int64
 5   TOURPURP     20739 non-null  int64
 6   N_STOPS      20739 non-null  int64
 7   N_TRIPS      20739 non-null  int64
 8   N_TRIPS_HBW  20739 non-null  int64
 9   N_TRIPS_HBO  20739 non-null  int64
 10  N_TRIPS_NHB  20739 non-null  int64
dtypes: int64(11)
memory usage: 1.7 MB


In [72]:
tour.head()

,TOURID,HHID,PERSONID,DTAZ,TOURMODE,TOURPURP,N_STOPS,N_TRIPS,N_TRIPS_HBW,N_TRIPS_HBO,N_TRIPS_NHB
0,0,50000,60000,22,1,1,0,2,2,0,0
1,1,50000,60001,4,1,1,0,2,2,0,0
2,2,50000,60001,10,2,2,0,2,0,2,0
3,3,50000,60002,20,1,1,0,2,2,0,0
4,4,50000,60002,20,1,2,1,3,0,2,1


In [73]:
# work (purpose=1) and non-work (purpose=2).
tour.TOURPURP.value_counts()

2    13175
1     7564
Name: TOURPURP, dtype: int64

In [74]:
# Get only work trips
df = tour[tour.TOURPURP == 1]
df.shape

(7564, 11)

In [75]:
# Merge data
df = df.merge(hh, on="HHID").merge(pp, on=('HHID', 'PERSONID'))
df.shape

(7564, 33)

In [76]:
# For looking up data in the skims matrix using Larch, need zero-based numbering
df["HOMETAZi"] = df["HOMETAZ"] - 1
df["DTAZi"] = df["DTAZ"] - 1 

In [77]:
df.head().T

,0,1,2,3,4
TOURID,0,1,3,7,10
HHID,50000,50000,50000,50001,50003
PERSONID,60000,60001,60002,60004,60006
DTAZ,22,4,20,25,25
TOURMODE,1,1,1,1,1
TOURPURP,1,1,1,1,1
N_STOPS,0,0,0,2,1
N_TRIPS_x,2,2,2,4,3
N_TRIPS_HBW_x,2,2,2,1,1
N_TRIPS_HBO_x,0,0,0,1,1


In [78]:
los_data = skims.get_rc_dataframe(df["HOMETAZi"], df["DTAZi"])
los_data

,AUTO_COST,AUTO_DIST,AUTO_TIME,BIKE_TIME,TRANSIT_FARE,TRANSIT_IVTT,TRANSIT_OVTT,WALK_DIST,WALK_TIME
0,0.588461,1.681318,5.043955,8.406591,0.0,0.000000,2.241758,1.681318,33.626365
1,1.925965,5.502758,16.508274,16.843315,2.5,1.535243,47.830632,3.368663,67.373260
2,0.621348,1.775280,5.325840,8.876400,0.0,0.000000,35.505600,1.775280,35.505600
3,0.480560,1.373029,9.119086,6.865143,0.0,0.000000,27.460572,1.373029,27.460572
4,0.480560,1.373029,9.119086,6.865143,0.0,0.000000,27.460572,1.373029,27.460572
...,...,...,...,...,...,...,...,...,...
7559,0.990705,2.830584,4.842947,8.820128,2.5,1.652554,21.496711,1.764026,35.280512
7560,1.066707,3.047736,7.201386,15.238678,2.5,3.746109,11.138607,3.047736,60.954713
7561,1.294075,3.697356,7.290231,18.486780,2.5,10.514265,33.910995,3.697356,73.947121
7562,1.066707,3.047736,7.201386,15.238678,2.5,3.746109,11.138607,3.047736,60.954713


In [79]:
df = df.join(los_data)
df.shape

(7564, 44)

### Model definition

In [80]:
# Define numbers as names for modes
DA,SR,Walk,Bike,Transit = 1,2,3,4,5
dfs = larch.DataFrames(co=df, 
                       alt_codes=[DA,SR,Walk,Bike,Transit],
                       alt_names=['DA','SR','Walk','Bike','Transit'],
                       ch_name='TOUR_MODE')

In [81]:
# Define model
m = larch.Model(dataservice=dfs)
m.title = "Work Tour Mode Choice"

In [82]:
# Define Utility function
m.utility_co[DA] = P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_COST
m.utility_co[SR] = (P.ASC_SR + P.InVehTime * X.AUTO_TIME 
                    + P.Cost * (X.AUTO_COST * 0.5) + P("LogIncome:SR") * X("log(INCOME)"))
m.utility_co[Walk] = (
        + P.ASC_Walk
        + P.NonMotorTime * X.WALK_TIME
        + P("LogIncome:Walk") * X("log(INCOME)")
)

m.utility_co[Bike] = (
        + P.ASC_Bike
        + P.NonMotorTime * X.BIKE_TIME
        + P("LogIncome:Bike") * X("log(INCOME)")
)

m.utility_co[Transit] = (
        + P.ASC_Transit
        + P.InVehTime * X.TRANSIT_IVTT
        + P.OutVehTime * X.TRANSIT_OVTT
        + P.Cost * X.TRANSIT_FARE
        + P("LogIncome:Transit") * X('log(INCOME)')
)

In [83]:
m.graph

In [84]:
# Nested logit model
Car = m.graph.new_node(parameter='Mu:Car', children=[DA,SR], name='Car')
NonMotor = m.graph.new_node(parameter='Mu:NonMotor', children=[Walk,Bike], name='NonMotor')
Motor = m.graph.new_node(parameter='Mu:Motor', children=[Car,Transit], name='Motor')

In [85]:
m.graph

In [86]:
m.choice_co_code = 'TOURMODE'

In [87]:
m.availability_co_vars = {
    DA: 'AGE >= 16',
    SR: 1,
    Walk: 'WALK_TIME < 60',
    Bike: 'BIKE_TIME < 60',
    Transit: 'TRANSIT_FARE>0',
}

## Model Estimation

In [88]:
m.load_data()

In [89]:
m.dataframes.choice_avail_summary()

,name,chosen,available
1,DA,6052.0,7564
2,SR,810.0,7564
3,Walk,196.0,4179
4,Bike,72.0,7564
5,Transit,434.0,4199
< Total All Alternatives >,,7564.0,


In [90]:
m.dataframes.data_co.statistics()

n    minimum     maximum     median  \
AUTO_COST     7564   0.194926    4.307963   1.009448   
AUTO_TIME     7564   0.930008   29.441527   7.615709   
BIKE_TIME     7564   2.784652   52.232051  13.586354   
TRANSIT_FARE  7564   0.000000    2.500000   2.500000   
TRANSIT_IVTT  7564   0.000000   12.166819   1.447685   
TRANSIT_OVTT  7564   0.759059  128.651674  37.049942   
WALK_TIME     7564  11.138607  208.928204  54.345418   
log(INCOME)   7564   7.590347   14.218053  10.671870   

                                                      histogram       mean  \
AUTO_COST     [[[<Element '{http://www.w3.org/1999/02/22-rdf...   1.206009   
AUTO_TIME     [[[<Element '{http://www.w3.org/1999/02/22-rdf...   8.222874   
BIKE_TIME     [[[<Element '{http://www.w3.org/1999/02/22-rdf...  15.982692   
TRANSIT_FARE  [[[<Element '{http://www.w3.org/1999/02/22-rdf...   1.387824   
TRANSIT_IVTT  [[[<Element '{http://www.w3.org/1999/02/22-rdf...   2.622771   
TRANSIT_OVTT  [[[<Element '{http://www.w3.org/1999/02/22-rdf...  37.661443   
WALK_TIME     [[[<Element '{http://www.w3.org/1999/02/22-rdf...  63.930768   
log(INCOME)   [[[<Element '{http://www.w3.org/1999/02/22-rdf...  10.841356   

                  stdev  zeros  positives  negatives  nonzero_minimum  \
AUTO_COST      0.754844      0       7564          0         0.194926   
AUTO_TIME      4.581342      0       7564          0         0.930008   
BIKE_TIME      9.297318      0       7564          0         2.784652   
TRANSIT_FARE   1.242379   3365       4199          0         2.500000   
TRANSIT_IVTT   3.380879   3365       4199          0         0.959322   
TRANSIT_OVTT  23.219383      0       7564          0         0.759059   
WALK_TIME     37.189270      0       7564          0        11.138607   
log(INCOME)    1.013010      0       7564          0         7.590347   

              nonzero_maximum  nonzero_mean  nonzero_stdev  
AUTO_COST            4.307963      1.206009       0.754844  
AUTO_TIME           29.441527      8.222874       4.581342  
BIKE_TIME           52.232051     15.982692       9.297318  
TRANSIT_FARE         2.500000      2.500000       0.000000  
TRANSIT_IVTT        12.166819      4.724610       3.264970  
TRANSIT_OVTT       128.651674     37.661443      23.219383  
WALK_TIME          208.928204     63.930768      37.189270  
log(INCOME)         14.218053     10.841356       1.013010

In [91]:
result = m.maximize_loglike(method='slsqp')

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
ASC_Bike,-0.258430,0.0,0.0,-inf,inf,0,,-0.258430
ASC_SR,1.422554,0.0,0.0,-inf,inf,0,,1.422554
ASC_Transit,6.753916,0.0,0.0,-inf,inf,0,,6.753916
ASC_Walk,8.621143,0.0,0.0,-inf,inf,0,,8.621143
Cost,-0.175666,0.0,0.0,-inf,inf,0,,-0.175666
InVehTime,-0.123722,0.0,0.0,-inf,inf,0,,-0.123722
LogIncome:Bike,-0.196942,0.0,0.0,-inf,inf,0,,-0.196942
LogIncome:SR,-0.193766,0.0,0.0,-inf,inf,0,,-0.193766
LogIncome:Transit,-0.557110,0.0,0.0,-inf,inf,0,,-0.557110
LogIncome:Walk,-0.522755,0.0,0.0,-inf,inf,0,,-0.522755


/home/danph/.pyenv/versions/miniconda3-4.7.10/envs/.dcm/lib/python3.9/site-packages/larch/model/optimization.py:306: UserWarning: slsqp may not play nicely with unbounded parameters
if you get poor results, consider setting global bounds with model.set_cap()
  warnings.warn( # infinite bounds # )


In [92]:
m.calculate_parameter_covariance()

In [93]:
m.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value
ASC_Bike,-0.258,1.34,-0.19,,0.00
ASC_SR,1.42,1.00,1.42,,0.00
ASC_Transit,6.75,2.06,3.27,**,0.00
ASC_Walk,8.62,1.14,7.57,***,0.00
Cost,-0.176,0.120,-1.47,,0.00
InVehTime,-0.124,0.0292,-4.24,***,0.00
LogIncome:Bike,-0.197,0.124,-1.59,,0.00
LogIncome:SR,-0.194,0.135,-1.43,,0.00
LogIncome:Transit,-0.557,0.169,-3.29,***,0.00
LogIncome:Walk,-0.523,0.100,-5.21,***,0.00


In [94]:
m.estimation_statistics()

<xmle.Elem 'div' with 1 children>

## Save and Report Model

In [95]:
report = larch.Reporter(title=m.title)

In [96]:
report << '# Parameter Summary' << m.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value
ASC_Bike,-0.258,1.34,-0.19,,0.00
ASC_SR,1.42,1.00,1.42,,0.00
ASC_Transit,6.75,2.06,3.27,**,0.00
ASC_Walk,8.62,1.14,7.57,***,0.00
Cost,-0.176,0.120,-1.47,,0.00
InVehTime,-0.124,0.0292,-4.24,***,0.00
LogIncome:Bike,-0.197,0.124,-1.59,,0.00
LogIncome:SR,-0.194,0.135,-1.43,,0.00
LogIncome:Transit,-0.557,0.169,-3.29,***,0.00
LogIncome:Walk,-0.523,0.100,-5.21,***,0.00


In [97]:
report.save(
    './exampville_mode_choice.html', 
    overwrite=True, 
    metadata=m,
)

'./exampville_mode_choice.html'